# Notebook notes

This notebook is for testing and training of CNN-based DQN

Each member of "options" list is a State99 

what happens after call step?

1. state of the player controlled by Agent is updated. It becomes exactly as one passed in "aciton" dictionary
2. states of all other players are updated
3. after all of the updates happened, *.step() calculates all options for the NEXT step (for pieces it holds at hand and in swap). These options, along with rewards they will trigger, are returned.

# Import libs

**setup autoreload**

In [1]:
%load_ext autoreload
%autoreload 2

**configure system path**

In [2]:
import os
import sys

# list directories where packages are stored
# note that the parent directory of te repo is added automatically
GYM_FOLDER = "gym-t99"

# get this notebook's current working directory
nb_cwd = os.getcwd()
# get name of its parent directory
nb_parent = os.path.dirname(nb_cwd)
# add packages to path
sys.path.insert(len(sys.path), nb_parent)
sys.path.insert(len(sys.path), os.path.join(nb_parent, GYM_FOLDER))

**import custom packages**

Use this code if you have not yet loaded gym_t99

In [3]:
import gym
registered = gym.envs.registration.registry.env_specs.copy()

import gym_t99
import t_net

pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


**import custom packages**

Use this code if you have already loaded gym_t99, but now you introduced changes and want to re-load it again

In [4]:
import imp

# this code removes environment from gym's registry
env_dict = gym.envs.registration.registry.env_specs.copy()
for env in env_dict:
    if env not in registered:
        print("Remove {} from registry".format(env))
        del gym.envs.registration.registry.env_specs[env]

imp.reload(gym_t99)

Remove basic-v0 from registry
Remove t99-v0 from registry
Remove t99sc-v0 from registry


<module 'gym_t99' from '/Users/ianconceicao/239AS_T99_RL_Agent/gym-t99/gym_t99/__init__.py'>

In [25]:
import imp
imp.reload(t_net)

<module 't_net' from '/Users/ianconceicao/239AS_T99_RL_Agent/t_net/__init__.py'>

**import out-of-the-box packages**

In [5]:
# import matplotlib
import matplotlib.pyplot as plt
# configure matplotlib
%matplotlib inline

import numpy as np
from copy import deepcopy

import glob
import re

import imageio

# import torch
import torch
from torch import nn

# configure torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Testground

In [6]:
custom_gym = gym.make('gym_t99:t99sc-v0', num_players=1, enemy=None)

In [7]:
custom_gym.reset()

In [9]:
frame = custom_gym.render(mode="debug")
# frame[0][-4, 2:6] = 1
print(frame[0])

[[10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  7  7  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  7  7  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0

# NN training

In [8]:
# initialize net
my_net = t_net.FCNet()
# initialize features
my_features = t_net.calculate_linear_features

In [ ]:
# initialize an enemy
my_npc = t_net.StandardNPC(my_net, device, my_features)
# assign the enemy to the environemnt
custom_gym.enemy = my_npc

In [9]:
agent = t_net.AgentSCFixedTarget(env=custom_gym,
                                 discount = 0.99,
                                 net = my_net,
                                 learning_rate = 0.001,
                                 criterion = nn.MSELoss(),
                                 device=device,
                                 features = my_features)

In [15]:
#Load pre-trained agent
#agent.load_state("agents/FC_net_transfer_learning.pth")

In [10]:
#Reset agent variables
agent.cumulative_rewards = [0]
agent.steps_per_episode = [0]
agent.episode = 0
agent.exploration_rate = 0
custom_gym.reset()
png_path = "png_saves/1"
gif_path = "gif_saves/1/episode1.gif"

In [13]:
agent.train(steps=200,image_save_path=png_path,episode_to_save=0)

calculating step 0
target net updated


In [18]:
#Get png names
pattern = os.path.join(png_path,"*.png")
png = glob.glob(pattern)
png.sort(key=lambda f: int(re.sub('\D', '', f)))

#Save into gif
images = []
for filename in png:
    images.append(imageio.imread(filename))

kargs = { 'fps': 7 }
imageio.mimsave(gif_path, images,'GIF', **kargs)

#Optimize gif
from pygifsicle import optimize
optimize(gif_path)